In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

# Load and preprocess the dataset
dataset_path = "./dataset/blackhole.csv"  # dataset path
data = pd.read_csv(dataset_path)

# Preprocessing
features = data.drop(columns=['category', 'label'])
labels = data['label']

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Further split training set into client datasets
num_clients = 5
client_data = np.array_split(X_train, num_clients)
client_labels = np.array_split(y_train, num_clients)

# Define a simple neural network for the Prototypical Network
def create_model(input_shape):
    model = Sequential([
        InputLayer(shape=(input_shape,)),  # Updated from input_shape to shape
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize a global model
global_model = create_model(X_train.shape[1])


2024-07-26 23:31:27.404207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 23:31:27.514450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 23:31:27.547049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 23:31:27.773412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 23:31:29.008569: W tensorflow/compiler/tf2

Federated Learning and Few-Shot Learning Implementation

In [2]:
# Federated learning simulation
def federated_training(global_model, client_data, client_labels, epochs=100):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        client_weights = []
        for i in range(num_clients):
            # Clone the global model
            local_model = tf.keras.models.clone_model(global_model)
            local_model.set_weights(global_model.get_weights())
            
            # Compile the cloned model
            local_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
            
            # Train on client's data
            local_model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
            
            # Collect the weights after training
            client_weights.append(local_model.get_weights())
        
        # Average the weights to update the global model
        new_weights = [np.mean([client_weight[layer] for client_weight in client_weights], axis=0) for layer in range(len(client_weights[0]))]
        global_model.set_weights(new_weights)

# Train the global model with federated learning
federated_training(global_model, client_data, client_labels, epochs=100)

# Evaluate on the test set
y_pred = (global_model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/100


I0000 00:00:1722016893.783202   66644 service.cc:146] XLA service 0x7ff50c005330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722016893.783243   66644 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-07-26 23:31:33.836570: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 23:31:34.053602: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1722016895.170149   66644 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
